In [ ]:
pip install llama-stack==0.0.36 llama-stack-client==0.0.35 nest_asyncio tavily-python

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.2/87.2 kB 2.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 223.2/223.2 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.5/184.5 kB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.0/78.0 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 22.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 34.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.4/75.4 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 28.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 1.3 MB/s eta 0:00:00
  Created wheel for fire: filename=fire-0.7.0-py3-none-any.whl size=114249 sha256=2fcf763d4346d8fad9cf990fe037b011c1793e6554334d5cde6207130e2efdc1
  Stored in directory: /root/

In [ ]:
# Install required packages
# !pip install llama-stack==0.0.36 llama-stack-client==0.0.35 nest_asyncio tavily-python

import nest_asyncio
nest_asyncio.apply()

import os
import asyncio
from typing import List, Optional, Dict
from dataclasses import dataclass
from datetime import datetime
from pathlib import Path
import traceback
from tavily import TavilyClient

from llama_stack_client import LlamaStackClient
from llama_stack_client.lib.agents.event_logger import EventLogger
from llama_stack_client.types import SamplingParams, UserMessage
from llama_stack_client.types.agent_create_params import AgentConfig

# Set your API key
os.environ["TOGETHER_API_KEY"] = "your_api_key_here"

# Constants
LLAMA_STACK_API_TOGETHER_URL = "https://llama-stack.together.ai"
LLAMA31_8B_INSTRUCT = "Llama3.1-8B-Instruct"

def save_to_markdown(content: str, filename: str, output_dir: str = "outputs"):
    """Save content to a markdown file with timestamp"""
    Path(output_dir).mkdir(parents=True, exist_ok=True)
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    full_filename = f"{filename}_{timestamp}.md"
    filepath = os.path.join(output_dir, full_filename)

    with open(filepath, "w", encoding="utf-8") as f:
        f.write(content)

    print(f"Saved to: {filepath}")
    return filepath

class BaseAgent:
    def __init__(self, role_instructions: str):
        self.client = LlamaStackClient(
            base_url=LLAMA_STACK_API_TOGETHER_URL,
        )
        self.role_instructions = role_instructions

    def create_agent(self):
        agent_config = AgentConfig(
            model=LLAMA31_8B_INSTRUCT,
            instructions=self.role_instructions,
            enable_session_persistence=True,
        )

        agent = self.client.agents.create(
            agent_config=agent_config,
        )
        self.agent_id = agent.agent_id
        session = self.client.agents.sessions.create(
            agent_id=agent.agent_id,
            session_name=f"session_{datetime.now().strftime('%Y%m%d_%H%M%S')}",
        )
        self.session_id = session.session_id

    async def execute_turn(self, content: str) -> str:
        try:
            response = self.client.agents.turns.create(
                agent_id=self.agent_id,
                session_id=self.session_id,
                messages=[
                    UserMessage(content=content, role="user"),
                ],
                stream=True,
            )

            full_response = ""
            for chunk in response:
                if hasattr(chunk, 'event') and hasattr(chunk.event, 'payload'):
                    payload = chunk.event.payload

                    if hasattr(payload, 'text_delta_model_response') and payload.text_delta_model_response:
                        full_response += payload.text_delta_model_response
                        print(payload.text_delta_model_response, end="", flush=True)

                    elif payload.event_type == "step_complete" and hasattr(payload, 'step_details'):
                        if hasattr(payload.step_details, 'inference_model_response'):
                            if not full_response and payload.step_details.inference_model_response.content:
                                full_response = payload.step_details.inference_model_response.content

            return full_response.strip()

        except Exception as e:
            print(f"\nError in execute_turn: {str(e)}")
            raise

class ResearchAgent(BaseAgent):
    def __init__(self):
        super().__init__(
            role_instructions="""You are a research agent. Your role is to gather and synthesize information on given topics.
            When you receive a topic, you will:
            1. Extract key search terms
            2. Plan appropriate research queries
            3. Organize and summarize findings
            4. Identify key insights and data points"""
        )
        self.tavily_client = TavilyClient(api_key="your_api_key_here")  # Replace with your key

    async def conduct_research(self, topic: str) -> str:
        # First, extract main topic for search
        prompt = f"""Given this article topic: "{topic}"
        What would be the most relevant search query to gather information about this?
        Provide just the search query, nothing else."""

        search_query = await self.execute_turn(prompt)
        print(f"\n🔍 Searching for: {search_query}")

        try:
            # Simple direct search using Tavily
            search_results = self.tavily_client.search(search_query)

            # Save raw search results first
            raw_results = "# Raw Search Results\n\n"
            raw_results += f"Search Query: {search_query}\n\n"
            raw_results += "## Full Results:\n"
            for idx, result in enumerate(search_results["results"], 1):
                raw_results += f"\n### Result {idx}\n"
                raw_results += f"Title: {result['title']}\n"
                raw_results += f"URL: {result['url']}\n"
                raw_results += f"Content:\n{result['content']}\n"
                raw_results += f"Score: {result['score']}\n"
                raw_results += "-" * 80 + "\n"

            save_to_markdown(
                content=raw_results,
                filename="raw_search_results",
                output_dir="article_outputs"
            )

            # Process results for the writer
            research_data = "## Processed Research Findings\n\n"
            research_data += f"Based on search query: {search_query}\n\n"

            for result in search_results["results"][:3]:
                research_data += f"### {result['title']}\n"
                research_data += f"Source: {result['url']}\n"
                research_data += f"{result['content']}\n\n"

            # Save processed research findings
            save_to_markdown(
                content=research_data,
                filename="processed_research_data",
                output_dir="article_outputs"
            )

            return research_data

        except Exception as e:
            error_msg = f"Error in web search: {str(e)}"
            print(error_msg)
            save_to_markdown(
                content=error_msg,
                filename="research_error_log",
                output_dir="article_outputs"
            )
            return "Research data unavailable."

class WriterAgent(BaseAgent):
    def __init__(self):
        super().__init__(
            role_instructions="""You are an expert content writer. Your role is to:
            1. Create engaging and informative content based on provided research
            2. Maintain consistent tone and style
            3. Include relevant examples and data points
            4. Structure content with clear sections and transitions
            5. Optimize for readability and engagement"""
        )

    async def write_article(self, article_idea: str, research_data: str) -> str:
        prompt = f"""Please write a comprehensive article based on this topic and research:

        ARTICLE TOPIC:
        {article_idea}

        RESEARCH DATA:
        {research_data}

        Write an engaging and informative article that:
        1. Incorporates key insights from the research
        2. Uses specific examples and data points
        3. Has clear sections and structure
        4. Maintains professional tone

        Format your response as:
        <ARTICLE>
        Your article content here with proper sections and formatting
        </ARTICLE>"""

        response = await self.execute_turn(prompt)

        # Save writer's output
        save_to_markdown(
            content=response,
            filename="writer_draft",
            output_dir="article_outputs"
        )

        return response

class EditorAgent(BaseAgent):
    def __init__(self):
        super().__init__(
            role_instructions="""You are an expert content editor. Your role is to:
            1. Review and improve article structure and flow
            2. Enhance clarity and readability
            3. Check for consistency in tone and style
            4. Verify SEO optimization
            5. Suggest specific improvements
            6. Calculate readability and SEO scores"""
        )

    async def edit_article(self, article_content: str, original_requirements: str) -> str:
        prompt = f"""Please edit and improve the following article:
        Title: {original_requirements}
        Content: {article_content}

        Provide your response in the following format:

        <EDITED_CONTENT>
        Your edited article here...
        </EDITED_CONTENT>

        <IMPROVEMENTS>
        - Improvement 1
        - Improvement 2
        - Improvement 3
        </IMPROVEMENTS>

        <SCORES>
        SEO: <score 0-100>
        Readability: <score 0-100>
        </SCORES>"""

        response = await self.execute_turn(prompt)

        # Save editor's output
        save_to_markdown(
            content=response,
            filename="editor_revision",
            output_dir="article_outputs"
        )

        return response

async def run_article_pipeline(topic: str = "AI in Healthcare"):
    print("🚀 Starting Article Pipeline")

    try:
        output_dir = "article_outputs"
        Path(output_dir).mkdir(parents=True, exist_ok=True)

        # Initialize agents
        researcher = ResearchAgent()
        writer = WriterAgent()
        editor = EditorAgent()

        print("\n📝 Creating agents...")
        researcher.create_agent()
        writer.create_agent()
        editor.create_agent()

        # Save initial article idea
        article_idea = """**Article Title:** "The Future of Healthcare: How AI is Revolutionizing Patient Care"
        **Target Audience:** Healthcare professionals, patients, and tech-savvy individuals
        **Key Points:**
        1. Introduction to AI in healthcare
        2. AI-assisted diagnosis
        3. Personalized medicine
        4. Predictive analytics
        5. Ethical considerations
        **Word Count:** 800-1000 words"""

        save_to_markdown(
            content=article_idea,
            filename="initial_outline",
            output_dir=output_dir
        )

        print("\n🔍 Conducting research...")
        research_data = await researcher.conduct_research(article_idea)

        print("\n✍️ Writing article...")
        article_draft = await writer.write_article(article_idea, research_data)

        print("\n📋 Editing article...")
        edited_result = await editor.edit_article(article_draft, article_idea)

        # Save final report
        final_report = f"""# Article Generation Report

## Pipeline Stages:
1. Initial Outline
2. Research Findings
3. Writer's Draft
4. Editor's Revision

## Files Generated:
- Initial Outline
- Research Data
- Writer's Draft
- Editor's Revision

Generated on: {datetime.now().strftime("%Y-%m-%d %H:%M:%S")}
        """

        save_to_markdown(
            content=final_report,
            filename="generation_report",
            output_dir=output_dir
        )

        return edited_result

    except Exception as e:
        error_log = f"""# Error Log
Time: {datetime.now().strftime("%Y-%m-%d %H:%M:%S")}
Error: {str(e)}
Details: {traceback.format_exc()}
        """
        save_to_markdown(
            content=error_log,
            filename="error_log",
            output_dir=output_dir
        )
        raise

# Run the pipeline
if __name__ == "__main__":
    print("Starting pipeline execution...")
    asyncio.run(run_article_pipeline())

Starting pipeline execution...
🚀 Starting Article Pipeline

📝 Creating agents...
Saved to: article_outputs/initial_outline_20241027_060223.md

🔍 Conducting research...
"AI in healthcare applications, AI-assisted diagnosis, personalized medicine, predictive analytics in healthcare, artificial intelligence ethics in healthcare, healthcare AI integration, machine learning in patient care, AI for disease prevention, AI-powered medical diagnosis accuracy, healthcare data analytics AI."
🔍 Searching for: "AI in healthcare applications, AI-assisted diagnosis, personalized medicine, predictive analytics in healthcare, artificial intelligence ethics in healthcare, healthcare AI integration, machine learning in patient care, AI for disease prevention, AI-powered medical diagnosis accuracy, healthcare data analytics AI."
Saved to: article_outputs/raw_search_results_20241027_060230.md
Saved to: article_outputs/processed_research_data_20241027_060230.md

✍️ Writing article...
<ARTICLE>

**The Future